In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser


In [10]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system',
         """당신은 주어진 문서에 대해서 답변하는 QA 챗봇입니다. 
         모르면 모른다고 답변해 주세요
         답변에 출처를 남겨주세요
         답변에 대해서 추가적으로 요약해 주세요
         한국어로 답변해 주세요
         """),
         MessagesPlaceholder(variable_name='chat_history'),
         ('user',
        """
            #Qusetion:
            {question}
        """
)

    ]
)
llm = ChatOpenAI(model='gpt-4o-mini')

chain = prompt|llm|StrOutputParser()

In [11]:
# 세션을 저장하는 히스토리
store = {}

def get_session_history(session_ids):
    print(f"[대화세션ID]: {session_ids}")
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]



In [29]:
ChatMessageHistory

langchain_core.chat_history.InMemoryChatMessageHistory

In [12]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

In [23]:
chain_with_history.invoke({"question": "안녕? 내이름은 재호야"},
                           config={"configurable": {"session_id": "asd"}})

# chain_with_history.invoke({"question": "안녕? 내이름은 재호야"},
#                          config={"configurable": {"session_id": "asd"}})

[대화세션ID]: asd


'안녕하세요, 재호님! 반갑습니다. \n\n요약: 재호라는 이름을 말씀하셨습니다.'

In [24]:
chain_with_history.invoke({"question": "내 이름이 뭐라고?"},
                           config={"configurable": {"session_id": "asd"}})

[대화세션ID]: asd


'당신의 이름은 재호입니다. \n\n요약: 이름이 재호라고 하셨습니다.'

In [26]:
query = """
langchain으로 RAG Multi-trun 대화를 구축하고 있습니다.
def get_session_history(session_ids):
    print(f"[대화세션ID]: {session_ids}")
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]
이 함수에 대해서 설명해주세요

"""
response = chain_with_history.invoke({"question": query},
                           config={"configurable": {"session_id": "asd"}})

[대화세션ID]: asd


In [27]:
print(response)

주어진 `get_session_history(session_ids)` 함수는 LangChain을 사용하여 RAG(정보 검색과 생성) Multi-turn 대화를 구현하는 데 필요한 대화 세션 히스토리를 관리하는 기능을 수행합니다. 함수의 각 부분에 대한 설명은 다음과 같습니다.

1. **함수 정의**: `def get_session_history(session_ids)`:
   - `session_ids`는 대화 세션을 식별하는 고유한 문자열 또는 식별자입니다.

2. **로그 출력**: `print(f"[대화세션ID]: {session_ids}")`:
   - 현재 처리 중인 세션 ID를 콘솔에 출력합니다. 이는 디버깅이나 로그 기록 목적으로 유용합니다.

3. **세션 존재 확인**: `if session_ids not in store:`:
   - `store`라는 데이터 구조(주로 딕셔너리 형태일 가능성이 높음)에서 해당 세션 ID가 존재하는지 확인합니다.

4. **새로운 세션 초기화**: 
   - `store[session_ids] = ChatMessageHistory()`:
     - 만약 해당 세션 ID가 `store`에 없다면, 새로운 `ChatMessageHistory` 객체를 생성하여 `store`에 추가합니다. 이 객체는 해당 대화 세션의 메시지 히스토리를 저장하는 역할을 합니다.

5. **히스토리 반환**: `return store[session_ids]`:
   - 마지막으로, 해당 세션 ID에 대한 `ChatMessageHistory` 객체를 반환합니다. 이 객체를 통해 대화의 과거 메시지에 접근할 수 있습니다.

**목적**: 
- 이 함수는 여러 사용자 또는 대화 세션을 관리하는 환경에서 각 세션의 대화 히스토리를 효율적으로 추적할 수 있도록 돕습니다. 이를 통해 사용자는 대화의 흐름을 유지하고, 이전 메시지에 쉽게 접근할 수 있게 됩니다.

요약: `get_session_history` 함수는 주어진 세션 ID에 대한 대화

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharcterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableWithMessageHistory, RunnablePassthrough
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.output_parsers import StrOutputParser


In [1]:
from langchain_teddynote.document_loaders import PDFParser
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

loader = PDFParser(file_path='./data/stroke paper draft4.pdf', multimodal=llm)

docs = loader.load()

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?